# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x24b756e0ba8>)

In [4]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89176,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
89177,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
89178,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
89179,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,213.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [16]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, max_iter=10000, positive=True, precompute=True,
      selection='random')

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 143.99183348823468
Test MAE: 147.08894148293928


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.044054272748459467
Day -6 NewCases 0.38818943395674793
Day -5 NewCases 0.267987985568403
Day -4 NewCases 0.060603381610973964
Day -3 NewCases 0.024635984695023948
Day -2 NewCases 0.08923571788822025
Day -1 NewCases 0.19920207572483511
Day -26 C6_Stay at home requirements 5.773239160619521
Day -22 C2_Workplace closing 2.8975369086913227
Day -21 C2_Workplace closing 10.74535993710082
Intercept 26.96089126501346


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [20]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [21]:
os.getcwd()

'C:\\Users\\marti\\Desktop\\Projects\\Pandemic-Prize\\covid-xprize\\covid_xprize\\examples\\predictors\\linear'

In [22]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="../../../validation/data/2020-09-30_historical_ip.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 68.90672517815139
2020-08-02: 78.75962279046016
2020-08-03: 80.54279905477833
2020-08-04: 87.0935887074165
2020-08-05: 93.59258062852393
2020-08-06: 104.97895215458959
2020-08-07: 136.29639033408034
2020-08-08: 151.44636536090846
2020-08-09: 160.8150866202873
2020-08-10: 169.85801839988343
2020-08-11: 180.6294319956761
2020-08-12: 197.83009449421263
2020-08-13: 219.72698291681203
2020-08-14: 236.20863056584778
2020-08-15: 249.24998969929118
2020-08-16: 261.7101890920532
2020-08-17: 276.27840844709937
2020-08-18: 294.6321983761607
2020-08-19: 314.3604236374357
2020-08-20: 331.89975385766235
2020-08-21: 347.6169227889191
2020-08-22: 363.22682927851645
2020-08-23: 380.4893327791255
2020-08-24: 399.8246381552569
2020-08-25: 419.72096325592
2020-08-26: 438.6707519904341
2020-08-27: 451.0275120517328
2020-08-28: 478.9654654450156
2020-08-29: 502.7740303827118
2020-08-30: 525.0608758664021
2020-08-31: 546.7158382751627

Predicting for Afghanistan__nan
20

2020-08-28: 413.54775685942536
2020-08-29: 426.58592404358143
2020-08-30: 443.2209112140717
2020-08-31: 460.7170937678342

Predicting for Belgium__nan
2020-08-01: 2730.1979632842913
2020-08-02: 2813.763529664165
2020-08-03: 2163.7824135899723
2020-08-04: 1686.213165881071
2020-08-05: 1573.5273311829505
2020-08-06: 1553.810694364145
2020-08-07: 2488.6502982665106
2020-08-08: 2619.9234474181076
2020-08-09: 2345.562331034678
2020-08-10: 2080.2388490107396
2020-08-11: 1992.7252959611476
2020-08-12: 2190.720223334238
2020-08-13: 2596.3940538243155
2020-08-14: 2695.254905073923
2020-08-15: 2578.8985212689336
2020-08-16: 2448.00021099647
2020-08-17: 2445.9623828926624
2020-08-18: 2618.7202918998605
2020-08-19: 2835.370117359391
2020-08-20: 2911.0265947342946
2020-08-21: 2873.6707363570126
2020-08-22: 2853.792996756292
2020-08-23: 2907.027841436503
2020-08-24: 3044.555838821467
2020-08-25: 3185.935476045042
2020-08-26: 3255.3803117950956
2020-08-27: 3271.949176070933
2020-08-28: 3298.168661415

2020-08-01: 136.84581659927403
2020-08-02: 137.7988468709695
2020-08-03: 137.00313087671987
2020-08-04: 157.15017898221737
2020-08-05: 150.75260027024171
2020-08-06: 149.0967600467018
2020-08-07: 197.52810492409355
2020-08-08: 214.27669474047792
2020-08-09: 226.6386096183896
2020-08-10: 237.7598477910824
2020-08-11: 242.3864129537243
2020-08-12: 257.67441333102124
2020-08-13: 285.37183646459385
2020-08-14: 304.9894989063766
2020-08-15: 320.5429677579667
2020-08-16: 333.1023112414685
2020-08-17: 345.5368636231106
2020-08-18: 364.2677028114136
2020-08-19: 387.04318225422895
2020-08-20: 407.3227429426853
2020-08-21: 424.8775756573214
2020-08-22: 451.51871626577224
2020-08-23: 473.56951425780875
2020-08-24: 495.9233747225572
2020-08-25: 519.1652944633563
2020-08-26: 541.5278348624686
2020-08-27: 564.7910784888365
2020-08-28: 590.3725329841441
2020-08-29: 615.2278951006581
2020-08-30: 640.2677923289082
2020-08-31: 665.5137763548657

Predicting for Brazil__nan
2020-08-01: 35419.08539693116
2

2020-08-19: 308.9705068840412
2020-08-20: 327.1823555519127
2020-08-21: 342.9283260390576
2020-08-22: 379.4082656763324
2020-08-23: 406.19285530427584
2020-08-24: 429.3297801830998
2020-08-25: 451.68044656404817
2020-08-26: 473.24455433996656
2020-08-27: 504.1920385802301
2020-08-28: 536.2666894957445
2020-08-29: 565.5084362899388
2020-08-30: 592.4161324990364
2020-08-31: 618.5258172443096

Predicting for Cote d'Ivoire__nan
2020-08-01: 77.85248032050114
2020-08-02: 94.84791695842748
2020-08-03: 104.099804354474
2020-08-04: 94.38355945010962
2020-08-05: 89.38559159675441
2020-08-06: 107.68684438649612
2020-08-07: 145.85174136088068
2020-08-08: 166.8820043281226
2020-08-09: 176.36129360762646
2020-08-10: 177.47203217050023
2020-08-11: 183.9065232872797
2020-08-12: 203.9073464863522
2020-08-13: 230.32489548581248
2020-08-14: 249.98333673283722
2020-08-15: 262.0433173498274
2020-08-16: 270.6360119747928
2020-08-17: 283.4158880320139
2020-08-18: 303.34407946123736
2020-08-19: 325.8011642569

2020-08-09: 3363.1110874225037
2020-08-10: 3713.0700675836542
2020-08-11: 3519.3479323052857
2020-08-12: 3285.757760013455
2020-08-13: 3454.8408126128948
2020-08-14: 3576.5876720438832
2020-08-15: 3803.1050181334263
2020-08-16: 3945.814429341883
2020-08-17: 3885.3191794018785
2020-08-18: 3845.0626281694704
2020-08-19: 3936.8609115091067
2020-08-20: 4074.126913344433
2020-08-21: 4236.5438123963295
2020-08-22: 4330.133826338703
2020-08-23: 4344.230579907205
2020-08-24: 4374.018933645067
2020-08-25: 4464.006754809845
2020-08-26: 4591.46495520665
2020-08-27: 4709.103396455851
2020-08-28: 4795.1968292029305
2020-08-29: 4849.016166709912
2020-08-30: 4914.34243785056
2020-08-31: 5011.810349708254

Predicting for Germany__nan
2020-08-01: 15565.304231851434
2020-08-02: 11773.172872895571
2020-08-03: 10949.305356165763
2020-08-04: 15965.602651952364
2020-08-05: 15683.349691824662
2020-08-06: 10846.611745832264
2020-08-07: 13866.58160396314
2020-08-08: 13326.4733213026
2020-08-09: 14209.539675856

...


Predicting for United States__Louisiana
2020-08-01: 2558.1268874662915
2020-08-02: 1362.1773485730325
2020-08-03: 1557.6339310374024
2020-08-04: 2112.300222754389
2020-08-05: 2875.4532550318186
2020-08-06: 1854.5458776786909
2020-08-07: 2182.694261817366
2020-08-08: 1910.1978386173812
2020-08-09: 2078.1285331500953
2020-08-10: 2461.9202165341685
2020-08-11: 2613.623868842917
2020-08-12: 2390.968930290319
2020-08-13: 2389.167878595007
2020-08-14: 2349.616289461911
2020-08-15: 2501.32109295358
2020-08-16: 2711.5102745015092
2020-08-17: 2781.9629105974223
2020-08-18: 2735.8622809188187
2020-08-19: 2726.2304715839277
2020-08-20: 2759.89402929989
2020-08-21: 2882.3502726882043
2020-08-22: 3024.618943297424
2020-08-23: 3091.284065745197
2020-08-24: 3104.9430521680224
2020-08-25: 3127.790584607837
2020-08-26: 3189.285370471774
2020-08-27: 3289.3230085017944
2020-08-28: 3394.616264569045
2020-08-29: 3463.224135736203
2020-08-30: 3506.840224960721
2020-08-31: 3556.25830648868

Predicting for U

2020-08-15: 923.2625067907852
2020-08-16: 893.9105214844808
2020-08-17: 878.5789642686393
2020-08-18: 920.3418428790806
2020-08-19: 993.7267200124531
2020-08-20: 1045.9680158925921
2020-08-21: 1063.1786481144457
2020-08-22: 1060.6615154036265
2020-08-23: 1071.3776087701874
2020-08-24: 1112.0804117291862
2020-08-25: 1166.452870127184
2020-08-26: 1209.1522544315833
2020-08-27: 1226.6968482945751
2020-08-28: 1240.4611864132405
2020-08-29: 1264.0726177286854
2020-08-30: 1303.8679432158303
2020-08-31: 1349.6474656374196

Predicting for United States__Nebraska
2020-08-01: 1330.17255044436
2020-08-02: 1322.1199137397682
2020-08-03: 1561.757866844711
2020-08-04: 1796.7303607907431
2020-08-05: 1441.8158107632257
2020-08-06: 1053.491120571913
2020-08-07: 1400.2525647863424
2020-08-08: 1559.8661182223343
2020-08-09: 1747.1691663853876
2020-08-10: 1790.3906524949634
2020-08-11: 1609.1728499882963
2020-08-12: 1517.7671608706112
2020-08-13: 1656.0801407252177
2020-08-14: 1801.0621951021838
2020-08-1

2020-08-24: 7635.678405906122
2020-08-25: 7929.449264085939
2020-08-26: 8159.91400170012
2020-08-27: 8266.863093314852
2020-08-28: 8312.406880719205
2020-08-29: 8401.029624755372
2020-08-30: 8590.676634892769
2020-08-31: 8828.0282349506

Predicting for United States__Rhode Island
2020-08-01: 861.3992657015269
2020-08-02: 492.81476625186593
2020-08-03: 1040.9401051652173
2020-08-04: 1716.6321242183683
2020-08-05: 1132.5550534642366
2020-08-06: 769.4141683982318
2020-08-07: 878.3134721092433
2020-08-08: 935.9199247907216
2020-08-09: 1290.3856644196555
2020-08-10: 1476.729932608497
2020-08-11: 1259.2171388720674
2020-08-12: 1107.225621610635
2020-08-13: 1125.3247712853997
2020-08-14: 1243.4537936549802
2020-08-15: 1441.749649447899
2020-08-16: 1512.688405030346
2020-08-17: 1431.562912301987
2020-08-18: 1370.0435340955446
2020-08-19: 1396.3016818466224
2020-08-20: 1498.1378457276817
2020-08-21: 1615.526094361959
2020-08-22: 1659.4489110353386
2020-08-23: 1637.920460596424
2020-08-24: 1626.

2020-08-16: 4365.877248797856
2020-08-17: 4208.532973818745
2020-08-18: 4249.119711921447
2020-08-19: 4524.462762315782
2020-08-20: 4766.102515740357
2020-08-21: 4860.372593584282
2020-08-22: 4833.946999951258
2020-08-23: 4805.173364397522
2020-08-24: 4896.662396735072
2020-08-25: 5090.811463699995
2020-08-26: 5266.535388344196
2020-08-27: 5353.760064036155
2020-08-28: 5383.327724093908
2020-08-29: 5425.280621687262
2020-08-30: 5535.351638624455
2020-08-31: 5693.525249111232

Predicting for United States__West Virginia
2020-08-01: 829.9526082555417
2020-08-02: 949.7696445798135
2020-08-03: 1083.4001080054704
2020-08-04: 1013.4927604187817
2020-08-05: 840.5133075166373
2020-08-06: 659.68619451273
2020-08-07: 925.8967234284755
2020-08-08: 1073.1780600937757
2020-08-09: 1149.7523590452736
2020-08-10: 1106.1417821996793
2020-08-11: 1005.3612993198888
2020-08-12: 985.7317918546298
2020-08-13: 1111.2381427417304
2020-08-14: 1218.7836023841903
2020-08-15: 1269.341895495805
2020-08-16: 1250.34

2020-08-30: 545.267706757942
2020-08-31: 567.9580662965348

Predicting for Zimbabwe__nan
2020-08-01: 140.61104827252353
2020-08-02: 138.26075091182886
2020-08-03: 142.1479517585227
2020-08-04: 193.9264194421586
2020-08-05: 190.51803659494539
2020-08-06: 164.85582825166162
2020-08-07: 207.0190616053062
2020-08-08: 222.5059409874887
2020-08-09: 243.7960895111115
2020-08-10: 268.2603264697254
2020-08-11: 272.05106499518575
2020-08-12: 277.6396207729732
2020-08-13: 300.37118689270443
2020-08-14: 320.5488824380828
2020-08-15: 342.4671955371108
2020-08-16: 360.9831927319385
2020-08-17: 372.54916600152006
2020-08-18: 386.69643914208586
2020-08-19: 406.8089204899259
2020-08-20: 428.1929171822265
2020-08-21: 449.65631056576177
2020-08-22: 468.445794442883
2020-08-23: 484.9464967849322
2020-08-24: 503.12613422239843
2020-08-25: 524.1450141885684
2020-08-26: 546.4385228839337
2020-08-27: 568.5122174499627
2020-08-28: 589.1797120673693
2020-08-29: 609.1945993836807
2020-08-30: 630.3376313269725
20

In [23]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,68.906725
214,Aruba,NaN,2020-08-02,78.759623
215,Aruba,NaN,2020-08-03,80.542799
216,Aruba,NaN,2020-08-04,87.093589
217,Aruba,NaN,2020-08-05,93.592581


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [24]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip ../../../validation/data/2020-09-30_historical_ip.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [25]:
!head predictions/2020-08-01_2020-08-04.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
os.getcwd()

'C:\\Users\\marti\\Desktop\\Projects\\Pandemic-Prize\\covid-xprize\\covid_xprize\\examples\\predictors\\linear'

In [27]:
%cd C:/Users/marti/Desktop/Projects/Pandemic-Prize/covid-xprize

C:\Users\marti\Desktop\Projects\Pandemic-Prize\covid-xprize


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [41]:
%cd ../../../../

C:\Users\marti\Desktop\Projects\Pandemic-Prize\covid-xprize


In [29]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [42]:
%cd covid_xprize/examples/predictors/linear

C:\Users\marti\Desktop\Projects\Pandemic-Prize\covid-xprize\covid_xprize\examples\predictors\linear


In [36]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="./predictions/val_4_days.csv")

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to ./predictions/val_4_days.csv
Done!
All good!


## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [37]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

Generating predictions from 2021-01-01 to 2021-01-31...
Saved predictions to predictions/val_1_month_future.csv
Done!
All good!
Wall time: 1.76 s


## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [38]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

Start date: 2020-12-09
End date: 2021-06-07


In [39]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

Saved scenario to predictions/180_days_future_scenario.csv


### Check it

In [40]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")

Generating predictions from 2020-12-09 to 2021-06-07...
Saved predictions to predictions/val_6_month_future.csv
Done!
All good!
Wall time: 3min 35s
